<a href="https://colab.research.google.com/github/yckamra/MovieLens25m-Recommender/blob/main/Movie_Dictionary_Creation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
print("Installing project dependencies...")
!pip install numpy pandas scipy scikit-learn # Basic libraries we want
!pip install implicit # Installs implicit library for matrix factorization
#!pip install torch
print("Dependencies installed successfully.")

Installing project dependencies...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 69.0 MB/s eta 0:00:00
Dependencies installed successfully.


In [2]:
from google.colab import auth
import zipfile
import os
import pickle
import pandas as pd
import numpy as np
import implicit
import json

In [3]:
auth.authenticate_user()

!gcloud config set project movielens-recommender-451017

!gsutil cp gs://movielens-data/data.zip /content/

Updated property [core/project].
Copying gs://movielens-data/data.zip...
==> NOTE: You are downloading one or more large file(s), which would
run significantly faster if you enabled sliced object downloads. This
feature is enabled by default but requires that compiled crcmod be
installed (see "gsutil help crcmod").

| [1 files][262.6 MiB/262.6 MiB]                                                
Operation completed over 1 objects/262.6 MiB.                                    


TMDb dataset: https://www.kaggle.com/datasets/tmdb/tmdb-movie-metadata?select=tmdb_5000_movies.csv

Big TMDB dataset: https://www.kaggle.com/datasets/asaniczka/tmdb-movies-dataset-2023-930k-movies/data

MovieLens dataset: https://www.kaggle.com/datasets/garymk/movielens-25m-dataset

In [4]:
!ls -lh /content/

total 263M
-rw-r--r-- 1 root root 263M Jun  1 17:56 data.zip
drwxr-xr-x 1 root root 4.0K May 29 14:01 sample_data


In [5]:
zip_path = "/content/data.zip" # File name
extract_to = "/content/data/"  # Where to extract files

# Create directory if it doesn't exist
os.makedirs(extract_to, exist_ok=True)

# Unzip the file
with zipfile.ZipFile(zip_path, "r") as zip_ref:
    zip_ref.extractall(extract_to)

print("Extraction complete! Files are in:", extract_to)
!ls -lh /content/data/

Extraction complete! Files are in: /content/data/
total 8.0K
drwxr-xr-x 4 root root 4.0K Jun  1 17:56 data
drwxr-xr-x 3 root root 4.0K Jun  1 17:56 __MACOSX


In [6]:
!ls -lh /content/data/data/ml-25m

total 1.1G
-rw-r--r-- 1 root root 416M Jun  1 17:56 genome-scores.csv
-rw-r--r-- 1 root root  18K Jun  1 17:56 genome-tags.csv
-rw-r--r-- 1 root root 1.4M Jun  1 17:56 links.csv
-rw-r--r-- 1 root root 2.9M Jun  1 17:56 movies.csv
-rw-r--r-- 1 root root 647M Jun  1 17:56 ratings.csv
-rw-r--r-- 1 root root  11K Jun  1 17:56 README.txt
-rw-r--r-- 1 root root  38M Jun  1 17:56 tags.csv


In [7]:
genome_scores_CSV = "/content/data/data/ml-25m/genome-scores.csv"
genome_tags_CSV = "/content/data/data/ml-25m/genome-tags.csv"
links_CSV = "/content/data/data/ml-25m/links.csv"
movies_CSV = "/content/data/data/ml-25m/movies.csv"
ratings_CSV = "/content/data/data/ml-25m/ratings.csv"
tags_CSV = "/content/data/data/ml-25m/tags.csv"

In [27]:
class Movie:
  # Items in self:
  #     ID
  #     name
  #     genre_list
  #     tag_list
  #     number_of_ratings -- total amount this movies has been rated
  #     sum_of_ratings -- summation of all the ratings from all users for this movie
  #     year
  #     imdbId
  #     tmdbId
  #     actors_list
  #     tmdb_tag_list

  def __init__(self):
    self.ID = None
    self.title = None
    self.genre_list = []
    self.tag_list = []
    self.number_of_ratings = 0
    self.sum_of_ratings = 0
    self.year = None
    self.imdbId = None
    self.tmdbId = None
    self.actors_list = []
    self.tmdb_tag_list = []

  def set_tmdb_tag_list(self, tags):
    self.tmdb_tag_list = tags

  def add_to_tmdb_tag_list(self, tags):
    self.tmdb_tag_list = self.tmdb_tag_list + tags

  def get_tmdb_tag_list(self):
    return self.tmdb_tag_list

  def set_actors_list(self, actors_list):
    self.actors_list = actors_list

  def add_actor_to_actors_list(self, actor):
    self.actors_list.append(actor)

  def get_actors_list(self):
    return self.actors_list

  def set_imdbId(self, imbdId):
    self.imbdId = imbdId

  def get_imdbId(self):
    return self.imdbId

  def set_tmdbId(self, tmdbId):
    self.tmdbId = tmdbId

  def get_tmdbId(self):
    return self.tmdbId

  def set_year(self, year):
    self.year = year

  def get_year(self):
    return self.year

  def get_tag_list(self):
    return self.tag_list

  def add_tag_to_tag_list(self, tag):
    self.tag_list.append(tag)

  def get_ID(self):
    return self.ID

  def set_ID(self, ID):
    self.ID = ID

  def set_title(self, title):
    self.title = title

  def get_title(self):
    return self.title

  def set_genre_list(self, genre_list):
    self.genre_list = genre_list

  def add_genre_to_genre_list(self, genre):
    self.genre_list.append(genre)

  def get_genre_list(self):
    return self.genre_list

  def get_number_of_ratings(self):
    return self.number_of_ratings

  def set_number_of_ratings(self, number):
    self.number_of_ratings = number

  def add_to_number_of_ratings(self):
    self.number_of_ratings += 1

  def get_sum_of_ratings(self):
    return self.sum_of_ratings

  def set_sum_of_ratings(self, number):
    self.sum_of_ratings = number

  def add_rating_to_sum_of_ratings(self, rating):
    self.sum_of_ratings += rating

  def calculate_average_rating(self):
    if self.number_of_ratings == 0:
      return 0
    else:
      return self.sum_of_ratings / self.number_of_ratings

In [28]:
# Load ratings data
ratings_path = ratings_CSV
ratings_df = pd.read_csv(ratings_path)

print(ratings_df)

# Load movies data
movies_path = movies_CSV
movies_df = pd.read_csv(movies_path)

print(movies_df)

# Load tags data
tags_path = tags_CSV
tags_df = pd.read_csv(tags_path)

print(tags_df)

# Load links data
links_path = links_CSV
links_df = pd.read_csv(links_path)

print(links_df)

          userId  movieId  rating   timestamp
0              1      296     5.0  1147880044
1              1      306     3.5  1147868817
2              1      307     5.0  1147868828
3              1      665     5.0  1147878820
4              1      899     3.5  1147868510
...          ...      ...     ...         ...
25000090  162541    50872     4.5  1240953372
25000091  162541    55768     2.5  1240951998
25000092  162541    56176     2.0  1240950697
25000093  162541    58559     4.0  1240953434
25000094  162541    63876     5.0  1240952515

[25000095 rows x 4 columns]
       movieId                               title  \
0            1                    Toy Story (1995)   
1            2                      Jumanji (1995)   
2            3             Grumpier Old Men (1995)   
3            4            Waiting to Exhale (1995)   
4            5  Father of the Bride Part II (1995)   
...        ...                                 ...   
62418   209157                           

In [29]:
def parse_genres(genre_string):
  genre_list = genre_string.split('|')
  return genre_list

def parse_year_from_title(title): # Split title of movie from the year of release from the MovieLens data
  length_of_title = len(title)

def create_all_movies_dictionary(movies_df, ratings_df, tags_df, links_df): # TODO: SPLIT year from title and CREATE genre one-hot encoded vector
  all_movies_dictionary = {}
  for index, row in ratings_df.iterrows():
    if row["movieId"] not in all_movies_dictionary:
      all_movies_dictionary[row["movieId"]] = Movie()
    all_movies_dictionary[row["movieId"]].add_to_number_of_ratings()
    all_movies_dictionary[row["movieId"]].add_rating_to_sum_of_ratings(row["rating"])
  for index, row in movies_df.iterrows():
    if row["movieId"] not in all_movies_dictionary:
      print("movie not rated")
      all_movies_dictionary[row["movieId"]] = Movie()
      all_movies_dictionary[row["movieId"]].set_title(row["title"])
      all_movies_dictionary[row["movieId"]].set_ID(row["movieId"])
      all_movies_dictionary[row["movieId"]].set_genre_list(parse_genres(row["genres"]))
  for index, row in tags_df.iterrows():
    if row["movieId"] not in all_movies_dictionary:
      print("movie not rated")
      all_movies_dictionary[row["movieId"]] = Movie()
    all_movies_dictionary[row["movieId"]].add_tag_to_tag_list(row["tag"])
  for index, row in links_df.iterrows():
    if row["movieId"] not in all_movies_dictionary:
      print("movie not rated")
      all_movies_dictionary[row["movieId"]] = Movie()
    if pd.isna(row["imdbId"]) or row["imdbId"] == '':
      # do nothing
      pass
    else:
      all_movies_dictionary[row["movieId"]].set_imdbId(row["imdbId"])
    if pd.isna(row["tmdbId"]) or row["tmdbId"] == '':
      # do nothing
      pass
    else:
      all_movies_dictionary[row["movieId"]].set_tmdbId(row["tmdbId"])

  return all_movies_dictionary

In [30]:
allMovies = create_all_movies_dictionary(movies_df, ratings_df, tags_df, links_df)

In [31]:
the_ID = 6
print(f"movieId: {allMovies[the_ID].get_ID()}")
print(f"title: {allMovies[the_ID].get_title()}")
print(f"genre_list: {allMovies[the_ID].get_genre_list()}")
print(f"average_rating: {allMovies[the_ID].calculate_average_rating()}")
print(f"total_number_of_ratings: {allMovies[the_ID].get_number_of_ratings()}")
print(f"MovieLens_tags: {allMovies[the_ID].get_tag_list()}")

movieId: 6
title: Heat (1995)
genre_list: ['Action', 'Crime', 'Thriller']
average_rating: 3.854908898649748
total_number_of_ratings: 24588
MovieLens_tags: ['imdb top 250', 'great acting', 'realistic action', 'suspense', 'Al Pacino', 'atmospheric', 'bank robbery', 'crime', 'great acting', 'realistic action', 'Robert De Niro', 'suspense', 'tense', 'Val Kilmer', 'bank job', 'bank robbery', 'crime', 'dialogue', 'heist', 'heist movie', 'long', 'Al Pacino', 'atmospheric', 'realistic action', 'Robert De Niro', 'bank robbery', 'Los Angeles', 'realistic action', 'Al Pacino', 'atmospheric', 'Michael Mann', 'realistic action', 'Robert De Niro', 'visceral', 'atmospheric', 'bleak', 'slick', 'somber', 'tense', 'visceral', 'witty', 'bank job', 'betrayal', 'cat and mouse', 'chase', 'crime epic', 'criminal mastermind', 'cult film', 'detective', 'ex-con', 'gang', 'heist', 'heist movie', 'honor', 'loner', 'los angeles', 'murder', 'neo-noir', 'obsession', 'one last job', 'robbery', 'suspense', 'thief', 'a

In [32]:
!ls -lh /content/data/data/TMDb-data

total 44M
-rw-r--r-- 1 root root  39M Jun  1 17:56 tmdb_5000_credits.csv
-rw-r--r-- 1 root root 5.5M Jun  1 17:56 tmdb_5000_movies.csv


In [33]:
tmdb_credits_CSV = "/content/data/data/TMDb-data/tmdb_5000_credits.csv"
tmdb_movies_CSV = "/content/data/data/TMDb-data/tmdb_5000_movies.csv"

In [34]:
tmdb_credits_path = tmdb_credits_CSV
tmdb_credits_df = pd.read_csv(tmdb_credits_path)

#print(tmdb_credits_df.head())

tmdb_movies_path = tmdb_movies_CSV
tmdb_movies_df = pd.read_csv(tmdb_movies_path)

#print(tmdb_movies_df.head())

In [35]:
def parse_json_column_to_names(json_str):

  if pd.isna(json_str) or not isinstance(json_str, str):
      return [] # Return empty list for missing or non-string values
  try:
      # Parse the JSON string into a Python list of dictionaries
      list_of_dicts = json.loads(json_str)
      # Extract the 'name' from each dictionary in the list
      return [d['name'] for d in list_of_dicts if 'name' in d]
  except json.JSONDecodeError:
      print(f"Warning: Could not decode JSON string: {json_str[:50]}...") # Print a snippet
      return [] # Return empty list for malformed JSON

In [36]:
tmdb_movies_df['parsed_genres'] = tmdb_movies_df['genres'].apply(parse_json_column_to_names)
tmdb_movies_df['parsed_keywords'] = tmdb_movies_df['keywords'].apply(parse_json_column_to_names)

In [37]:
print(tmdb_movies_df.head())

      budget                                             genres  \
0  237000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   
1  300000000  [{"id": 12, "name": "Adventure"}, {"id": 14, "...   
2  245000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   
3  250000000  [{"id": 28, "name": "Action"}, {"id": 80, "nam...   
4  260000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   

                                       homepage      id  \
0                   http://www.avatarmovie.com/   19995   
1  http://disney.go.com/disneypictures/pirates/     285   
2   http://www.sonypictures.com/movies/spectre/  206647   
3            http://www.thedarkknightrises.com/   49026   
4          http://movies.disney.com/john-carter   49529   

                                            keywords original_language  \
0  [{"id": 1463, "name": "culture clash"}, {"id":...                en   
1  [{"id": 270, "name": "ocean"}, {"id": 726, "na...                en   
2  [{"id": 470, "nam

In [38]:
def create_tmdbId_to_movieId_dictionary(links_df):
  dict = {}
  for index, row in links_df.iterrows():
    if pd.isna(row["tmdbId"]) or row["tmdbId"] == '':
      pass
    else:
      dict[row["tmdbId"]] = row["movieId"]
  return dict

def populate_movies_with_tmdb_tags(tmdb_movies_df, allMovies, tmdbId_to_movieId_dictionary): # If a movie is not in MovieLens, we should add it and give it a movieId
  miss_count = 0
  for index, row in tmdb_movies_df.iterrows():
    if row["id"] not in tmdbId_to_movieId_dictionary:
      miss_count += 1
      pass
    else:
      allMovies[tmdbId_to_movieId_dictionary[row["id"]]].set_tmdb_tag_list(row["parsed_keywords"])
  print(miss_count)

In [39]:
tmdbId_to_movieId_dictionary = create_tmdbId_to_movieId_dictionary(links_df)
populate_movies_with_tmdb_tags(tmdb_movies_df, allMovies, tmdbId_to_movieId_dictionary)

205


In [40]:
print(allMovies[1].get_tmdb_tag_list())

['jealousy', 'toy', 'boy', 'friendship', 'friends', 'rivalry', 'boy next door', 'new toy', 'toy comes to life']


In [41]:
print(tmdb_movies_df["parsed_keywords"].iloc[2])

['spy', 'based on novel', 'secret agent', 'sequel', 'mi6', 'british secret service', 'united kingdom']


In [60]:
def popularity_based_recommendation(movie_dictionary):
  list_of_movies = []
  list_of_popularity = []
  for key in movie_dictionary:
    list_of_movies.append(movie_dictionary[key].get_ID())
    list_of_popularity.append(movie_dictionary[key].calculate_average_rating())
  combined = zip(list_of_popularity, list_of_movies)
  sorted_pairs = sorted(combined, reverse=True)
  sorted_popularity, sorted_movies = zip(*sorted_pairs)
  return sorted_movies # Returns the movie IDs in descending order (most popular first)

def weighted_rating_popularity_based_recommendation(movie_dictionary, threshold): # Weighted Rating: WR = (v/(v+m))R + (m/(v+m))C, threshold should be ratings_df.groupby('movieId').size().quantile(0.80) so higher threshold is typically advised (ie. 20)
  number_of_ratings = 0
  sum_of_ratings = 0
  for key in movie_dictionary:
    if movie_dictionary[key].get_number_of_ratings() != 0:
      number_of_ratings += movie_dictionary[key].get_number_of_ratings()
      sum_of_ratings += movie_dictionary[key].get_sum_of_ratings()

  global_mean_rating = sum_of_ratings / number_of_ratings
  list_of_movies = []
  list_of_weighted_popularity = []

  for key in movie_dictionary:
    list_of_movies.append(movie_dictionary[key].get_ID()) # Add movie ID to list in order to return back to user
    curr_movie_num_ratings = movie_dictionary[key].get_number_of_ratings() # Get the current movie's number of ratings
    min_num_of_votes_threshold = threshold # The number of ratings a movie needs to be considered and not just averaged out
    curr_movie_average_rating = movie_dictionary[key].calculate_average_rating()
    weighted_rating = ((curr_movie_num_ratings / (curr_movie_num_ratings + min_num_of_votes_threshold)) * curr_movie_average_rating) + ((min_num_of_votes_threshold / (curr_movie_num_ratings + min_num_of_votes_threshold)) * global_mean_rating)
    list_of_weighted_popularity.append(weighted_rating)

  combined = zip(list_of_weighted_popularity, list_of_movies)
  sorted_pairs = sorted(combined, reverse=True)
  sorted_weighted_popularity, sorted_movies = zip(*sorted_pairs)

  return sorted_movies # Returns the movie IDs in descending order (most popular first)

In [66]:
ID = 170705

movies_recommended = popularity_based_recommendation(allMovies)
print(movies_recommended)

movies_recommended_weighted = weighted_rating_popularity_based_recommendation(allMovies, 20)
print(movies_recommended_weighted)

print(allMovies[ID].get_title())
print(allMovies[ID].get_number_of_ratings())
print(allMovies[ID].get_title())
print(allMovies[ID].calculate_average_rating())

(209155, 208859, 208795, 208597, 208477, 208433, 208265, 208181, 208116, 208090, 208006, 207996, 207884, 207708, 207694, 207662, 207648, 207642, 207499, 207497, 207457, 207365, 207263, 207259, 207195, 207193, 207191, 207151, 207149, 207095, 207053, 206967, 206915, 206909, 206901, 206781, 206721, 206459, 206210, 206176, 205741, 205665, 205535, 205517, 205419, 205321, 205277, 204488, 204012, 204004, 203986, 203882, 203443, 203330, 203328, 203238, 203086, 202944, 202936, 202805, 202769, 202763, 202231, 202211, 202181, 202175, 202037, 201981, 201971, 201907, 201901, 201883, 201869, 201821, 201797, 201793, 201781, 201715, 201648, 201612, 201468, 201466, 201440, 201424, 201420, 201338, 201280, 201258, 201226, 201150, 201006, 200998, 200744, 200702, 200664, 200650, 200562, 200360, 200316, 200314, 200312, 200296, 200286, 200204, 200088, 200086, 200082, 200016, 199916, 199762, 199756, 199598, 199484, 199424, 199325, 199257, 199191, 198925, 198863, 198841, 198755, 198727, 198699, 198529, 198527,

In [69]:
def check_on_ratings(allMovies):
  number_of_missing_ratings = 0
  for key in allMovies:
    if allMovies[key].get_number_of_ratings() == 0:
      number_of_missing_ratings += 1
  return number_of_missing_ratings

print(f"movies without a rating: {check_on_ratings(allMovies)}")

movies without a rating: 3376


In [71]:
import pickle

# --- Saving ---
file_path = 'allMovies_data.pkl'
with open(file_path, 'wb') as f: # 'wb' means write in binary mode
    pickle.dump(allMovies, f)
print(f"Movie dictionary saved to {file_path}")

Movie dictionary saved to allMovies_data.pkl


In [75]:
# --- Loading ---
with open(file_path, 'rb') as f: # 'rb' means read in binary mode
    allMovies = pickle.load(f)
print(f"\nMovie dictionary loaded from {file_path}")

# Verify loaded data
print(f"Loaded movie '1' title: {allMovies[1].get_genre_list()}")


Movie dictionary loaded from allMovies_data.pkl
Loaded movie '1' title: ['Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy']
